In [85]:
import os
import tensorflow as tf
import matplotlib
from PIL import Image
import numpy as np
import random


In [86]:
def get_filenames_labels(data_path):
    
    # Initialize the output lists
    filenames = []
    labels = []
    
    # Iterate over the 6 different classes
    for i in range(6):
        class_dir = str(i) + '_signs'
        
        # Get all the filenames of the dir
        filenames_class = os.listdir(os.path.join(data_path, class_dir))
        filenames_class = [os.path.join(data_path, class_dir, f) for f in filenames_class if f.endswith('jpg')]
        # Add the labels of the corresponding class
        labels_class = [i] * len(filenames_class)
        labels = labels + labels_class
        filenames = filenames + filenames_class
        
    # Shuffle the filenames and labels randomly
    z = list(zip(filenames, labels))
    random.shuffle(z)
    filenames, labels = zip(*z)
    
    out = {'filenames': filenames, 'labels':labels}
    
    return out

In [87]:
train_raw_inputs = get_filenames_labels('data/SIGNS_processed/train_signs/')
dev_raw_inputs = get_filenames_labels('data/SIGNS_processed/dev_signs/')

In [90]:
def wrap_int64(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def wrap_bytes(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def convert_to_TFRecord(image_paths, labels, out_path, size=(64,64)):
    # Args:
    # image_paths   List of file-paths for the images.
    # labels        Class-labels for the images.
    # out_path      File-path for the TFRecords output file.    
    print("Converting: " + out_path)
    
    # Number of images. Used when printing the progress.
    num_images = len(image_paths)    
    
    # Open a TFRecordWriter for the output-file.
    with tf.python_io.TFRecordWriter(out_path) as writer:        
        
        # Iterate over all the image-paths and class-labels.
        for i, (path, label) in enumerate(zip(image_paths, labels)):
            if i % 100 == 0:
                print('Progress: {}/{} images converted'.format(i, len(labels)))
            # Load the image-file using matplotlib's imread function.
            img = Image.open(path)
            img = img.resize(size)
            img = np.array(img)
            # Convert the image to raw bytes.
            img_bytes = img.tostring()
            # Create a dict with the data we want to save in the
            # TFRecords file. You can add more relevant data here.
            data = {
                'image': wrap_bytes(img_bytes),
                'label': wrap_int64(label)
            }
            # Wrap the data as TensorFlow Features.
            feature = tf.train.Features(feature=data)
            # Wrap again as a TensorFlow Example.
            example = tf.train.Example(features=feature)
            # Serialize the data.
            serialized = example.SerializeToString()        
            # Write the serialized data to the TFRecords file.
            writer.write(serialized)
            
        print('Progress: {}/{} images converted'.format(i+1, len(labels)))

In [91]:
convert_to_TFRecord(
    image_paths=train_raw_inputs['filenames'],
    labels=train_raw_inputs['labels'],
    out_path='data/SIGNS_TFRecord_v2/train.tfrecords'
)

convert_to_TFRecord(
    image_paths=dev_raw_inputs['filenames'],
    labels=dev_raw_inputs['labels'],
    out_path='data/SIGNS_TFRecord_v2/dev.tfrecords'
)

Converting: data/SIGNS_TFRecord_v2/train.tfrecords
Progress: 0/864 images converted
Progress: 100/864 images converted
Progress: 200/864 images converted
Progress: 300/864 images converted
Progress: 400/864 images converted
Progress: 500/864 images converted
Progress: 600/864 images converted
Progress: 700/864 images converted
Progress: 800/864 images converted
Progress: 864/864 images converted
Converting: data/SIGNS_TFRecord_v2/dev.tfrecords
Progress: 0/216 images converted
Progress: 100/216 images converted
Progress: 200/216 images converted
Progress: 216/216 images converted


In [92]:
def _parse_function(serialized):
    features = \
    {
        'image': tf.FixedLenFeature((), tf.string, default_value=''),
        'label': tf.FixedLenFeature((), tf.int64, default_value=0)
    }
    # Parse the serialized data so we get a dict with our data.
    parsed_example = tf.parse_single_example(serialized=serialized,
                                             features=features)
    # Get the image as raw bytes.
    image_shape = tf.stack([64, 64, 3])
    image_raw = parsed_example['image']
    label = tf.cast(parsed_example['label'], tf.int32)
    
    # Decode the raw bytes so it becomes a tensor with type.
    image = tf.decode_raw(image_raw, tf.uint8)
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, image_shape)
    #image = tf.subtract(image, 116.779) # Zero-center by mean pixel
    image = tf.reverse(image, axis=[2]) # 'RGB'->'BGR'
    
    return image, label

In [93]:
filenames = tf.placeholder(tf.string, shape=[None])
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(_parse_function)  # Parse the record into tensors.
dataset = dataset.shuffle()
dataset = dataset.repeat()  # Repeat the input indefinitely.
dataset = dataset.batch(32).prefetch(32)
iterator = dataset.make_initializable_iterator()

# You can feed the initializer with the appropriate filenames for the current
# phase of execution, e.g. training vs. validation.
sess = tf.Session()

# Initialize `iterator` with training data.
training_filenames = ['data/SIGNS_TFRecord_v2/train.tfrecords']
sess.run(iterator.initializer, feed_dict={filenames: training_filenames})

image, label = sess.run(iterator.get_next())

sess.close()

In [82]:
image.shape

(32, 64, 64, 3)

In [94]:
label

array([2, 1, 4, 2, 1, 0, 0, 4, 4, 2, 0, 2, 2, 5, 0, 1, 0, 0, 5, 1, 4, 0,
       0, 2, 5, 2, 1, 5, 1, 4, 2, 5], dtype=int32)